
# 假设train_data包含所有特征和产量（yield）列
# 1. 提取特征和目标变量（产量）
# 注意：仅用训练数据计算相关性，避免测试数据泄露
features = all_features[:n_train]  # 训练集特征（与之前n_train定义一致）
yield_values = train_data['yield']  # 产量列（目标变量）

# 2. 合并特征和产量，计算皮尔逊相关系数
# 新增产量列到特征数据中，用于计算相关性
corr_data = pd.concat([features, yield_values], axis=1)

# 计算所有特征与产量的皮尔逊相关系数
corr_with_yield = corr_data.corr()['yield'].drop('yield')  # 排除产量与自身的相关系数

# 3. 打印所有特征的相关系数（按绝对值排序）
print("特征与产量的皮尔逊相关系数（按绝对值排序）：")
sorted_corr = corr_with_yield.abs().sort_values(ascending=False)
print(sorted_corr)

# 4. 筛选|r|>0.2的特征
selected_features = corr_with_yield[abs(corr_with_yield) > 0.2].index.tolist()

# 5. 结合领域知识调整（例如强制保留ndvi，即使其相关系数略低于0.2）
domain_features = ['ndvi']  # 根据农业知识确定的关键特征
for feat in domain_features:
    if feat not in selected_features:
        selected_features.append(feat)
        print(f"根据领域知识，强制保留特征：{feat}")

# 6. 更新特征集（训练集和测试集均只保留筛选后的特征）
all_features_selected = all_features[selected_features]

# 7. 输出最终保留的特征
print("\n筛选后保留的特征：")
print(all_features_selected.columns.tolist())


# 对所有特征标准化（基于训练集的均值和标准差，避免数据泄露）
numeric_features = all_features.columns
train_features_0 = all_features[:n_train][numeric_features]
# 计算训练集的均值和标准差
mean = train_features_0.mean()
std = train_features_0.std()
# 标准化所有特征（训练集和测试集用同一套参数）
all_features[numeric_features] = (all_features[numeric_features] - mean) / std

# 对所有特征标准化（基于训练集的均值和标准差，避免数据泄露）
numeric_features = all_features.columns
train_features_0 = all_features[:n_train][numeric_features]
# 计算训练集的均值和标准差
mean = train_features_0.mean()
std = train_features_0.std()
# 标准化所有特征（训练集和测试集用同一套参数）
all_features[numeric_features] = (all_features[numeric_features] - mean) / std

